In [1]:
#ELO RUN ONE

import os
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGCHAIN_API_KEY"] = "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"

import getpass

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")


if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


from langchain_community.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Define the storage directory
persist_directory = "./chroma_db"

# Initialize Chroma with persistence
catalog_store = Chroma(
    collection_name="Catalog",
    embedding_function=embeddings,
    persist_directory=persist_directory  # Specify where to store the database
)
courses_store = Chroma(
    collection_name="Courses",
    embedding_function=embeddings,
    persist_directory=persist_directory  # Specify where to store the database
)

print(f"ChromaDB is stored at: {persist_directory}")


from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
prompt = ChatPromptTemplate.from_template("You are an assistant meant to help Vanderbilt students with course registration and other relevant Vanderbilt information. Use the following pieces of retrieved context to answer the question. You will not go outside the scope of the retrieved context. If the answer to the question is not present in the retrieved context then reply \" That question is outside the scope of my knowledge: try rephrasing.\" Keep the answer as concise as possible while still answering the question completely.\n\nContext: {context} \n\nQuestion: {question} \n\nAnswer:")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()
assert len(example_messages) == 1
print(example_messages[0].content)


/var/folders/3p/s2hzq46x79zbl3zb0t91q2y00000gn/T/ipykernel_48184/4218127529.py:32: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  catalog_store = Chroma(


ChromaDB is stored at: ./chroma_db
You are an assistant meant to help Vanderbilt students with course registration and other relevant Vanderbilt information. Use the following pieces of retrieved context to answer the question. You will not go outside the scope of the retrieved context. If the answer to the question is not present in the retrieved context then reply " That question is outside the scope of my knowledge: try rephrasing." Keep the answer as concise as possible while still answering the question completely.

Context: (context goes here) 

Question: (question goes here) 

Answer:


In [2]:
# from langchain_community.document_loaders import PyPDFLoader

# # Load the PDF file
# pdf_loader = PyPDFLoader("Undergraduate_Catalog_2024-25.pdf")
# documents = pdf_loader.load()

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# # Initialize the text splitter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,  # Number of characters per chunk
#     chunk_overlap=300,  # Overlap between chunks
# )

# # Split the documents
# split_documents = text_splitter.split_documents(documents)
# print(f"Split catalog into {len(split_documents)} sub-documents.")

In [5]:
catalog = catalog_store.get()
courses = courses_store.get()

# for i, metadata in enumerate(courses['metadatas']):
#     print(f"\nDocument {i} metadata:")
#     print(metadata)

# for i, metadata in enumerate(catalog['metadatas']):
#     print(f"\nDocument {i} metadata:")
#     print(metadata)

{'ids': ['d65adc73-fac8-4a01-8d62-9e93df14b358',
  '41f6c81e-cdba-4308-9f05-90fc1eb9ec09',
  'c6141cce-1955-4d3b-8652-4c60cbd01255',
  'f1916b1b-f928-4e1d-89dc-91f2a0d89e29',
  '9254b0eb-fd48-4445-a644-fe146ac38cb2',
  '267fc3ce-8cf2-4834-8bf4-9d53534a0858',
  '4eec7944-5932-4bb3-b4b7-c46dcced3593',
  '5dba6c7a-3446-4e99-a825-9b889931a8f1',
  '8106c805-1645-4857-9696-3b3350304d76',
  'd012d5fe-ad7e-405b-bcc4-449971839040',
  'f7460302-fa6a-4720-96d2-22419d3f940e',
  '4852a66b-9517-4238-a5d3-f0c2c74e412a',
  'ca44ffff-853a-426e-b571-854f946085a5',
  '66f3ffa7-97a7-4d8c-8526-d3e362b01ec0',
  '8f7a1648-84e7-4367-b41b-66536231110e',
  '4235e38f-438a-48f0-b3c6-1a20d8338a5c',
  '4cbdcbf9-6161-460f-b027-23b8b3866959',
  '5b0906b8-2aaf-4ecf-8f97-98e0a9c4fd5a',
  '095ef740-e553-4a4b-a534-55c8091911ec',
  '330aa9f4-78d5-4ddc-b584-203447f153d4',
  '31074c88-244a-46fa-9f89-c93bbc80f4fd',
  '6af593d9-f108-4786-ab7a-97acab2b897b',
  '0b6638c7-4f71-4a28-abe4-9ca4241d402a',
  '0ca11dd2-3d01-40be-b3fe-

In [ ]:
#ELO RUN 2
import json
from langchain.schema import Document

# Load chunks from JSON file
with open("chunks.json", "r", encoding="utf-8") as file:
    chunk_data = json.load(file)  # Parse JSON file

# Convert JSON data into LangChain Document objects
catalog_documents = [
    Document(
        page_content=chunk["text"],  # Extract text content
        metadata={"id": chunk["id"], "page": chunk["metadata"]["page"], "source": "chunks.json"}
    )
    for chunk in chunk_data
]

print(f"Loaded {len(catalog_documents)} chunks from chunks.json.")

catalog_document_ids = catalog_store.add_documents(documents=catalog_documents)

print(catalog_document_ids[:3])

Loaded 1217 chunks from chunks.json.


In [111]:
#ELO RUN 3
import csv
from langchain_core.documents import Document

# DONT NEED TO RERUN
def read_csv_to_list(file_path):
    documents = []
    metadata = []
    ids = []
    
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for i, row in enumerate(reader):
            if i >= 1:
                page_content = ', '.join(row)  # Convert row to a string
                
                if len(row) > 1:
                    course_code = row[1].split(' -')[0]  # Strip everything including and after '-'
                    doc_metadata = {'course_code': course_code, 'subject_area': row[0]}  # Extract cleaned course 
                    metadata.append(doc_metadata)
                
                ids.append(f"doc_{i}")  # Assign a unique ID to each document
                
                # Create a Document object with page_content and metadata
                documents.append(Document(page_content=page_content, metadata=doc_metadata))

    return documents, metadata, ids

# Example usage
file_path = 'subject_courses.csv'  # Replace with your actual file path
course_documents, course_metadata, ids = read_csv_to_list(file_path)


In [ ]:
courses_store.add_documents(documents=course_documents, metadata=course_metadata)

We'll use LangGraph to tie together the retrieval and generation steps into a single application. This will bring a number of benefits:

We can define our application logic once and automatically support multiple invocation modes, including streaming, async, and batched calls.
We get streamlined deployments via LangGraph Platform.
LangSmith will automatically trace the steps of our application together.
We can easily add key features to our application, including persistence and human-in-the-loop approval, with minimal code changes.

In [132]:
from typing import TypedDict
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from typing_extensions import Annotated


class Search(TypedDict):
    """Search query."""
    query: Annotated[str, ..., "Search query to run."]

In [133]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    query: Search
    context: List[Document]
    answer: str

The state of our application controls what data is input to the application, transferred between steps, and output by the application. It is typically a TypedDict, but can also be a Pydantic BaseModel.

For a simple RAG application, we can just keep track of the input question, retrieved context, and generated answer:

In [134]:
def analyze_query(state: State):
    structured_llm = llm.with_structured_output(Search)
    query = structured_llm.invoke(state["question"])
    return {"query": query}

In [135]:
from langchain_core.tools import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    # Retrieve 4 documents from each collection
    catalog_docs = catalog_store.similarity_search(query, k=8)
    courses_docs = courses_store.similarity_search(query, k=4)

    # Label and retain metadata
    labeled_catalog_docs = [
    {
        "content": doc.page_content,
        "source": doc.metadata.get("source", "Undergraduate Catalog Information"),
        "metadata": doc.metadata  # Retain metadata
    }
    for doc in catalog_docs
    ]

    labeled_courses_docs = [
    {
        "content": doc.page_content,
        "source": doc.metadata.get("source", "Course Information"),
        "metadata": doc.metadata  # Retain metadata
    }
    for doc in courses_docs
    ]

    

    # Combine results
    retrieved_docs = labeled_catalog_docs + labeled_courses_docs

    serialized = "\n\n".join(
        (
            f"Source: {doc['source']}\n"
            f"Metadata: {doc['metadata']}\n"
            f"Content: {doc['content']}"
        )
        for doc in retrieved_docs
    )

    return serialized, retrieved_docs


In [136]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# step 3: generate a response using retrieved context
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]


     # Format retrieved documents with labels (like your original code)
    docs_content = "\n\n".join(doc.content for doc in tool_messages)

    system_message_content =(
        "You are an assistant meant to help Vanderbilt students with course registration and other relevant Vanderbilt information. Use the following pieces of retrieved context to answer the question. You will not go outside the scope of the retrieved context. If the answer to the question is not present in the retrieved context then reply \" That question is outside the scope of my knowledge: try rephrasing.\" Keep the answer as concise as possible while still answering the question completely.\n\n"
        f"Context:\n{docs_content}"
    
    )

    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}


retrieve stores retrieved docs from the vector store^

In [145]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode

from langgraph.graph import START, StateGraph, MessagesState

graph_builder = StateGraph(MessagesState)

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

calls the prompt and passes in the context and question to messages, and sends messages to the llm

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
input_message = "What are the requirements for the computer science major?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [140]:
from langgraph.checkpoint.memory import MemorySaver


#future, use redissaver to keep memory
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
input_message = "What are the requirements for the computer science major?\n\n Once you get the answer, look up more general information about the major."

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "what can i take as depth classes for this major?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = "what did i ask about above?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

In [79]:

from langchain_core.messages import SystemMessage
from langgraph.prebuilt import create_react_agent
system_message_content =(
        "You are an assistant meant to help Vanderbilt students with course registration and other relevant Vanderbilt information. Use the following pieces of retrieved context to answer the question. You will not go outside the scope of the retrieved context. If the answer to the question is not present in the retrieved context then reply \" That question is outside the scope of my knowledge: try rephrasing.\" Keep the answer as concise as possible while still answering the question completely.\n\n"
       
    
    )

system_message = SystemMessage(system_message_content)



In [143]:
from langgraph.prebuilt import create_react_agent

import logging
from langchain_core.callbacks import StdOutCallbackHandler

# logging.basicConfig(level=logging.DEBUG)

handler = StdOutCallbackHandler
memory = MemorySaver()
agent_executor = create_react_agent(llm, [retrieve], checkpointer=memory)

In [ ]:
print(agent_executor.nodes['agent'].__dict__)

In [ ]:
import inspect
print(inspect.getsource(retrieve))


In [ ]:
display(Image(agent_executor.get_graph().draw_mermaid_png()))

state object keeping track of everything. retrieve and generate are executed in order and modify the state object. retrieve is connected to the starting point

In [ ]:
config = {"configurable": {"thread_id": "def234"}}

input_message = (
    "What are the requirements for the computer science major?\n\n"
    "Once you get the answer, look up more general information about the major."
)

for event in agent_executor.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    
    config=config,
):
    event["messages"][-1].pretty_print()